In [144]:
### Importing ###
from numpy import exp, array, random, dot #imports linear algebra operations/types
import numpy as np #operations in numpy
from scipy import misc #misc. operations in scipy
import skimage
import math 
from __future__ import with_statement

In [145]:
### Get picture data ###
arr1 = np.divide(misc.imread('save_save_save_america02.png'),255) #read 1st image, divide by 255 for number between 0 and 1
arr1r = arr1[:,:,0] #reds
arr1g = arr1[:,:,1] #greens
arr1b = arr1[:,:,2] #blues
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [146]:
### Hyper-parameters ###
layerSize = 3 #number of filters in a layer
layerNum = 6 #number of layers
filterSize = 10 #size of filter
epochs = 10 #number of times trained
filters = [] #init filter
targets = np.ones((len(arr1r)*len(arr1r[0])*layerSize)) #target values

In [147]:
### Variables based on hyper-params ###
filterNum = layerSize**2 #number of filters per layer
totalFilterNum = filterNum*layerNum #total number of filters

In [148]:
### Set filters ###
def setFilters():
    global filters #Filters is a global variable
    filters = 0.01*random.random((layerNum, filterNum, filterSize, filterSize))-0.005 #Set filters to random 4d array
setFilters()
print(filters)

[[[[  9.84761786e-04   3.65796079e-03   1.77566749e-03 ...,
     -4.63605949e-03   2.06484573e-03   4.77499660e-03]
   [ -1.66766248e-03  -1.42706967e-03  -4.09463423e-03 ...,
     -1.88343455e-03  -4.69515239e-03  -5.33699619e-04]
   [  1.61104551e-03   3.52419318e-03  -2.70796577e-03 ...,
      6.89930779e-04  -2.96840715e-03  -3.74776415e-03]
   ..., 
   [  1.52576953e-03  -2.41273771e-03   4.27433922e-03 ...,
     -4.81207573e-05   6.37078881e-05  -3.40404065e-03]
   [  4.70303238e-03  -9.00960945e-04   2.74019227e-03 ...,
      1.51380008e-03   6.35774904e-04   2.57836346e-03]
   [  2.13893547e-04   2.58929852e-04   6.13310147e-04 ...,
     -3.67167838e-03   3.57853977e-03  -4.45667900e-03]]

  [[  1.10974009e-03  -2.23775272e-03  -1.04680113e-03 ...,
     -2.15079566e-03   3.63299912e-03  -4.97790223e-03]
   [  3.15577646e-03  -2.04934997e-03   4.15507904e-03 ...,
      1.10349073e-03   3.90898303e-03   3.68566777e-03]
   [  2.20580228e-03  -3.73496894e-04  -4.04464310e-03 ...,
 

In [149]:
### Helper functions ###
#Activation function (Leaky ReLU)
def activation(x, deriv=False, leak=-0.01):
    new_x=np.copy(array(x)) #Create a copy of x
    if deriv: #If you want to activate derivative
        #Calculation of derivative of leaky ReLU
        new_x[x > 0] = 1 
        new_x[x < 0] = leak
    #Calculation of leaky ReLU
    new_x[x < 0] *= leak
    new_x = x * (x > 0)
    return new_x
#Sum with no negatives (not used)
'''
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
'''
#Apply cross-correlation for 1 index
def apply(filterArr, img, row, col):
    value = 0 #Init value
    halfFil = int(filterSize/2) #Half of filterSize
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0)) #Pad image with 0's
    for x in range(filterSize): 
        for y in range(filterSize): #Apply over all filter
            value += padImg[x+row][y+col]*filterArr[x][y] #Add image multiplied by filter
    return value
#Softmax function
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0) #Calculate softmax
print(apply(filters[1][1],arr1r,0,0))

-0.00417263891409


In [150]:
### Apply filters ###
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])): #Apply over all image
            new_img[y][x] += activation(apply(filterArr,img,y,x)) #Add up all cross-correlations (with ReLU)
    return new_img

In [156]:
### Forward-propagation ###
def forward(img):
    outputLayers = [[img[:,:,0],img[:,:,1],img[:,:,2]]] #Get all the image 
    for num in range(layerNum):
        outputLayers.append([]) #Append a new layer
        for lay in range(layerSize):
            outputLayers[num+1].append(np.zeros((len(outputLayers[num][lay]), len(outputLayers[num][lay][0]))))
            # Add components of the layer
        #lay1 is the layer index you are at and lay2 is the layer index you are going to
        for lay1 in range(layerSize):
            for lay2 in range(layerSize):
                newImage = outputLayers[num+1][lay2] #Get refrence to next layer
                #print("S-O: ("+str(len(outputLayers))+", "+str(len(outputLayers[0]))+")")
                #print("S-F: ("+str(len(filters))+", "+str(len(filters[0]))+")")
                applyFilters(outputLayers[num][lay1], newImage, filters[num][layerSize*lay1+lay2]) 
                #Apply corresponding filter
                print(filterNum*num + layerSize*lay1+lay2)
#                 print(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten())) #Add flattened softmaxed array
    return outputLayers
out = forward(arr1)
print(out)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
[[array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.23529412,  0.23529412,  1.        ,  0.23529412,  0.23529412,
         0.23529412,  0.23529412,  1.    

In [118]:
def maxPool(size, img):
    new_img = np.zeros((int(len(img)/size), int(len(img[0])/size)))
    pad_img = np.pad(img, [(0, size-len(img)%size),(0, size-len(img[0])%size)], "constant", constant_values=(0))
    for x in range(math.ceil(len(img)/size)-1):
        for y in range(math.ceil(len(img[0])/size)-1):
            new_img[x][y] = np.amax(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
            print(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
    return new_img
print(maxPool(3, out[2]))

ValueError: Unable to create correctly shaped tuple from [(0, 3), (0, 3)]

In [164]:
### Error + Back-propagation ###
error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets)) # negative sum of log(out) times target- cross-entropy
print(error)
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
#         print("Filter S:"+str(len(filters)))
#         print(layerNum)
        for row in filters[layer-1][x]:
            c=0
            for w in row:
                new_grad[x][r][c]=-np.multiply(error,np.sum(np.multiply(np.multiply(
                    (array(out[layer-1])),
                    w),
                    grad[x][r][c])))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[layer-1][x][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

3695.01749526
6
5
4
3
2
1


In [167]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'ignore'}

In [168]:
for x in range(epochs):
    out = forward(arr1)
    error = 1-np.mean(out[len(out)-1])
    print(error)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))
    print(filters)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
0.999166666667
6
5
4
3
2
1
[[[[ -3.07380469e-03   4.49343945e-03  -2.17247523e-03 ...,
      1.27589981e-04  -3.61639259e-03  -1.94383950e-03]
   [  3.56482455e-03   2.16392668e-03  -1.32008009e-03 ...,
      4.84408574e-03   3.93988288e-03   5.20984866e-04]
   [ -3.20823257e-03   4.65192405e-04  -3.05508857e-03 ...,
      1.67912390e-04  -2.17501196e-03   8.22336887e-04]
   ..., 
   [  4.18547312e-03   2.15416901e-03   2.24346490e-03 ...,
      3.72764257e-05   4.77579332e-03   3.80699304e-03]
   [ -2.73019785e-03   2.44932745e-03   1.09138563e-03 ...,
     -2.49998271e-03  -1.73776896e-03  -3.94358171e-03]
   [  4.33677853e-03  -3.79412715e-03  -1.05838820e-03 ...,
     -4.51880854e-03   2.81300530e-03   2.31070250e-03]]

  [[ -3.24207038e-04  -7.85760096e-04   1.19337946e-03 ...,
      4.62765552e-04   3.89634821e-04  -1.35268472e-05